In [1]:
import itertools

import numpy as np
import IPython.display as disp
from sympy import simplify, collect, log, I, diff, eye, solve
from sympy.solvers.solveset import linear_eq_to_matrix
from symengine import expand, Symbol, conjugate, symbols, Matrix, linsolve, Add

# Normal Form of the Swift-Hohenberg Equation
In this notebook we determine the coefficients of the normal form of the Swift-Hohenberg equation.

In [2]:
# linear matrix
L = Matrix([[0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [-1, 0, -2, 0]])

In [3]:
L

[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]
[-1, 0, -2, 0]

In [4]:
# reversibility
R = eye(4)
for i in range(1, 3):
    R[2 * i - 1, 2 * i - 1] = -1

In [5]:
R

Matrix([
[1,  0, 0,  0],
[0, -1, 0,  0],
[0,  0, 1,  0],
[0,  0, 0, -1]])

In [6]:
# main variables
A = Symbol('A')
B = Symbol('B')

A_bar = Symbol('A_bar')  # conjugate(A)
B_bar = Symbol('B_bar')  # conjugate(B)

z_tilde = Matrix([A, B, A_bar, B_bar])

relations = {conjugate(A): A_bar, conjugate(B): B_bar,
             conjugate(A_bar): A, conjugate(B_bar): B}

In [7]:
quadratic_terms = set(np.product(el) for el in itertools.product(z_tilde, z_tilde))
cubic_terms = set(np.product(el) for el in itertools.product(z_tilde, z_tilde, z_tilde))

In [8]:
# integrals
c1 = A * A_bar
c2 = I * B / A + log(A)
c3 = I * (A * B_bar - A_bar * B) / 2

In [9]:
# vectors and matrices
zeta_0 = Matrix([1, I, -1, -I])
zeta_1 = Matrix([0, 1, 2 * I, -3])

# conjugate doesn't work on symengine Matrix
zeta_0_bar = Matrix([conjugate(el) for el in zeta_0])
zeta_1_bar = Matrix([conjugate(el) for el in zeta_1])

M = Matrix([[1, 0, 1, 0],
            [I, 1, -I, 1],
            [-1, 2 * I, -1,-2 * I],
            [-I, -3, I, -3]])

M_inv = M.inv()

L0 = M_inv * L * M

In [10]:
# polynomial vector
powers = [(0, 1, 2, 3) for _ in range(4)]
poly_powers = [el for el in itertools.product(*powers) if sum(el) in (2, 3)]
poly = 4 * [0]
quadratic_coefficients = []
cubic_coefficients = []
for idx in poly_powers:
    for j in range(4):
        coeff = Symbol('psi_{}'.format(''.join([str(el) for el in (j,) + idx])))
        poly[j] += coeff * np.product([x**i for x, i in zip(z_tilde, idx)])
        if sum(idx) == 2:
            quadratic_coefficients.append(coeff)
        elif sum(idx) == 3:
            cubic_coefficients.append(coeff)
psi = Matrix(poly)

## Reversibility
Now we make use of the reversibility of the system to determine relations between some of the unknown coefficients

In [11]:
# define a scaling to help collect together terms
epsilon = Symbol('epsilon')
scalings = {A: epsilon * A, B: epsilon * B,
            A_bar: epsilon * A_bar, B_bar: epsilon * B_bar}

In [12]:
psi_scaled = expand(psi.subs(scalings))

In [13]:
reversibility = {A: A_bar, B: -B_bar,
                 A_bar: A, B_bar: -B}
psiR_scaled = expand(psi_scaled.subs(reversibility))

In [14]:
psi_quadratic_relations = []
psi_cubic_relations = []

invariant_quadratic_terms = []
invariant_cubic_terms = []

for psi_l, psi_r in zip(psi_scaled, psiR_scaled):
    tmp_coll = collect(psi_l - psi_r, [epsilon**2, epsilon**3], evaluate=False)
    tmp_quad = collect(tmp_coll[epsilon._sympy_()**2], quadratic_terms, evaluate=False)
    tmp_cubic = collect(tmp_coll[epsilon._sympy_()**3], cubic_terms, evaluate=False)
    for term in quadratic_terms:
        try:
            psi_quadratic_relations.append(Add(tmp_quad[term._sympy_()]))
        except KeyError:
            invariant_quadratic_terms.append(term)
    for term in cubic_terms:
        try:
            psi_cubic_relations.append(Add(tmp_cubic[term._sympy_()]))
        except KeyError:
            invariant_cubic_terms.append(term)

In [15]:
quad_sol_rev = {}
psi_quadratic_relations = list(set(psi_quadratic_relations))
for eqn in psi_quadratic_relations:
    if eqn == 0:
        continue
    tmp_var = list(eqn.free_symbols)[0]
    tmp_sol = linsolve([eqn], [tmp_var])[0]
    for i in range(len(psi_quadratic_relations)):
        psi_quadratic_relations[i] = psi_quadratic_relations[i].subs({tmp_var: tmp_sol})
    quad_sol_rev.update({tmp_var: tmp_sol})

In [16]:
cubic_sol_rev = {}
psi_cubic_relations = list(set(psi_cubic_relations))
for eqn in psi_cubic_relations:
    if eqn == 0:
        continue
    tmp_var = list(eqn.free_symbols)[0]
    tmp_sol = linsolve([eqn], [tmp_var])[0]
    for i in range(len(psi_cubic_relations)):
        psi_cubic_relations[i] = psi_cubic_relations[i].subs({tmp_var: tmp_sol})
    cubic_sol_rev.update({tmp_var: tmp_sol})

In [17]:
#for i, p in enumerate(poly):
#    poly[i] = expand(p.subs(quad_sol_rev))
#for i, p in enumerate(poly):
#    poly[i] = expand(p.subs(cubic_sol_rev))

#psi = Matrix(poly)

In [18]:
#quad_coeffs = [coeff for coeff in psi.free_symbols if coeff in quadratic_coefficients]
#quadratic_coefficients = quad_coeffs

In [19]:
#cubic_coeffs = [coeff for coeff in psi.free_symbols if coeff in cubic_coefficients]
#cubic_coefficients = cubic_coeffs

In [20]:
psi_A = Matrix([diff(p, A) for p in poly])
psi_B = Matrix([diff(p, B) for p in poly])
psi_A_bar = Matrix([diff(p, A_bar) for p in poly])
psi_B_bar = Matrix([diff(p, B_bar) for p in poly])

In [21]:
# derivatives w.r.t. x
P1, P2, P1_bar, P2_bar = symbols('P1 P2 P1_bar P2_bar')
Q1, Q2, Q3, Q1_bar, Q2_bar, Q3_bar = symbols('Q1 Q2 Q3 Q1_bar Q2_bar Q3_bar')

P = P1 * c1 + P2 * c3
P = simplify(expand(P))
Q = Q1 * c1 + Q2 * c3 + Q3 * c1**2
Q = simplify(expand(Q))

P_bar = simplify(expand(conjugate(P).subs(relations)))
Q_bar = simplify(expand(conjugate(Q).subs(relations)))

In [22]:
# symengine doesn't support assumptions, so manually enforce the real/imagainary assumptions
assumptions = {conjugate(P1): P1_bar, conjugate(P2): P2_bar,
               conjugate(P1_bar): P1, conjugate(P2_bar): P2,
               conjugate(Q1): Q1_bar, conjugate(Q2): Q2_bar,
               conjugate(Q1_bar): Q1, conjugate(Q2_bar): Q2,
               conjugate(Q3): Q3_bar, conjugate(Q3_bar): Q3}

In [23]:
P = P.subs(assumptions)
Q = Q.subs(assumptions)
P_bar = P_bar.subs(assumptions)
Q_bar = Q_bar.subs(assumptions)

In [24]:
simplify(P_bar)

P1_bar*conjugate(A)*conjugate(A_bar) - I*P2_bar*conjugate(A)*conjugate(B_bar)/2 + I*P2_bar*conjugate(A_bar)*conjugate(B)/2

In [25]:
A_x = I * A + B + I * A * P
A_x = simplify(expand(A_x))
B_x = I * B + I * B * P + A * Q
B_x = simplify(expand(B_x))

A_bar_x = simplify(conjugate(A_x)).subs(relations).subs(assumptions)
B_bar_x = simplify(conjugate(B_x)).subs(relations).subs(assumptions)

In [26]:
A_x

I*A**2*A_bar*P1 - A**2*B_bar*P2/2 + A*A_bar*B*P2/2 + I*A + B

In [27]:
A_bar_x

-I*A*A_bar**2*P1_bar + A*A_bar*B_bar*P2_bar/2 - A_bar**2*B*P2_bar/2 - I*A_bar + B_bar

# Construct the Equations

We now construct the equations which determine the values of the unknown coefficients.

In [28]:
# construct LHS of equation
LHS = (zeta_0 + psi_A) * A_x + (zeta_1 + psi_B) * B_x
LHS += (zeta_0_bar + psi_A_bar) * A_bar_x + (zeta_1_bar + psi_B_bar) * B_bar_x

LHS = expand(LHS).subs(scalings)

In [29]:
# construct RHS of equation
z = z_tilde + psi
Mz = M * z
Mz = Mz.subs(scalings)

# linear part
RHS = L * z
RHS = expand(RHS).subs(scalings)

# nonlinear part
n2, n3 = symbols('n2 n3', real=True)
eps_terms = collect(Mz[0], [epsilon, epsilon**2], evaluate=False)
Mz = epsilon * eps_terms[epsilon._sympy_()] + epsilon**2 * eps_terms[epsilon._sympy_()**2]

# quadratic
n2_terms = n2 * Mz**2
n2_terms = collect(expand(n2_terms), [epsilon, epsilon**2], evaluate=False)
n2_terms = epsilon**2 * n2_terms[epsilon._sympy_()**2] + epsilon**3 * n2_terms[epsilon._sympy_()**3]

# cubic
n3_terms = n3 * Mz**3
n3_terms = collect(expand(n3_terms), [epsilon, epsilon**2], evaluate=False)
n3_terms = epsilon**3 * n3_terms[epsilon._sympy_()**3]

N = Matrix([0, 0, 0, n2_terms + n3_terms])
RHS += N

## Quadratic terms

In [30]:
# quadratic solution
out_l = [el.coeff(epsilon, 2) for el in LHS]
out_r = [el.coeff(epsilon, 2) for el in RHS]
quadratic_equations = []
for l, r in zip(out_l, out_r):
    tmp = collect(expand(l - r), quadratic_terms, evaluate=False)
    for term in quadratic_terms:
        quadratic_equations.append(Add(tmp[term._sympy_()]))  # incompatibility between sympy and symengine symbols

In [31]:
len(quadratic_equations)

40

In [32]:
# create the matrices
C2, C2r = linear_eq_to_matrix(quadratic_equations, [s._sympy_() for s in quadratic_coefficients])

In [33]:
sol = C2.LUsolve(C2r)

In [34]:
quadratic_subs = {key: val for key, val in zip(quadratic_coefficients, sol)}
for k, v in quadratic_subs.items():
    disp.display(k, v)

psi_00002

-28*n2/27

psi_10002

40*I*n2/27

psi_20002

...

2*I*n2/9

psi_22000

-4*n2/9

psi_32000

-8*I*n2/9

## Cubic terms
The matrix represenbting the LHS of the system of equations at cubic order is rank-deficient, so we can not use one of the `solve` procedures. Instead we calculate the null space of the LHS and multiply by the RHS to recover a system of equations involving the coefficients {P1, P2, Q1, Q2} (and their conjugates).

In [35]:
# cubic solution
out_l = [el.coeff(epsilon, 3) for el in LHS]
out_r = [el.coeff(epsilon, 3) for el in RHS]
cubic_equations = []
for l, r in zip(out_l, out_r):
    tmp = collect(expand(l - r).subs(quadratic_subs), cubic_terms, evaluate=False)
    for term in cubic_terms:
        cubic_equations.append(Add(tmp[term._sympy_()]))  # incompatibility betwen sympy and symengine symbols

In [36]:
cubic_equations[70]

(-3/2)*P2 + psi_00210 + 2*psi_20210 + I*psi_30210 + psi_31110 + (-52/27)*n2**2

In [37]:
free_coefficients = [P1, P2, Q1, Q2, P1_bar, P2_bar, Q1_bar, Q2_bar]

In [38]:
# create the matrices
C3, C3r = linear_eq_to_matrix(cubic_equations, [s._sympy_() for s in cubic_coefficients])

In [39]:
kern = C3.T.nullspace()

In [40]:
M_kern = Matrix([list(k) for k in kern])

In [41]:
system = simplify(M_kern * C3r)

In [42]:
sol3_free = solve(system, free_coefficients)

In [43]:
for k, v in sol3_free.items():
    disp.display(k, simplify(v))

P1

-37*n2**2/72 - 9*n3/16

P2

-92*n2**2/81

Q1

...

-92*n2**2/81

Q1_bar

-5*n2**2/6 - 3*n3/4

Q2_bar

-23*n2**2/36 - 3*n3/8